In [89]:
import itertools
import json
import math
import time

import pandas as pd
import plwn
import networkx as nx
from networkx.algorithms.cycles import simple_cycles
from networkx.algorithms.lowest_common_ancestors import lowest_common_ancestor
from networkx.algorithms.minors import contracted_nodes
import numpy as np
from tqdm import tqdm


tqdm.pandas()

## Load wordnet to graph

In [2]:
# plwn.download()

In [3]:
wn = plwn.load('./data/default_model')

In [4]:
#wn.to_graphml(out_file='./data/graph_synset.xml', graph_type='synset')

In [5]:
G = nx.read_graphml('./data/graph_synset.xml')

## Describe graph

In [6]:
print(f'Nodes: {len(G.nodes)}')
print(f'Edges: {len(G.edges)}')

Nodes: 340647
Edges: 1438540


## Load SimLex999 dataset

In [7]:
simlex = pd.read_csv('./data/MSimLex999_Polish.txt', sep='\t', header=None)
simlex.columns = ['id', 'word1', 'word2', 'similarity', 'relatedness']
simlex

,id,word1,word2,similarity,relatedness
0,1,stary,nowy,0.43,7.29
1,2,bystry,inteligentny,8.86,9.71
2,3,ciężki,trudny,4.86,7.29
3,4,szczęśliwy,radosny,8.14,8.86
4,5,łatwy,męczący,0.43,6.43
...,...,...,...,...,...
994,995,dołączyć,zdobyć,0.43,2.29
995,996,wysyłać,uczestniczyć,0.00,0.86
996,997,zbierać,uczestniczyć,0.00,0.71
997,998,pochłonąć,wycofać,0.00,0.57


## Create word: id json

In [8]:
unique_words = set(list(simlex['word1'].unique()) + list(simlex['word2'].unique()))

len(list(unique_words))

1139

In [9]:
# ids_dict = {}
# for word in tqdm(unique_words):
#     synset_id = None
#     try:
#         synset_id = wn.synset(word, plwn.PoS.noun, 1).to_dict()['id']
#     except plwn.exceptions.SynsetNotFound:
#         pass
#     if not synset_id:
#         try:
#             synset_id = wn.synset(word, plwn.PoS.verb, 1).to_dict()['id']
#         except plwn.exceptions.SynsetNotFound:
#             pass
#     if not synset_id:
#         try:
#             synset_id = wn.synset(word, plwn.PoS.adjective, 1).to_dict()['id']
#         except plwn.exceptions.SynsetNotFound:
#             pass
#     if not synset_id:
#         try:
#             synset_id = wn.synset(word, plwn.PoS.adverb, 1).to_dict()['id']
#         except plwn.exceptions.SynsetNotFound:
#             pass
#     
#     if synset_id:
#         ids_dict[word] = synset_id

In [10]:
#with open('data/ids_dict.json', 'w', encoding='utf8') as f:
#    json.dump(ids_dict, f, ensure_ascii=False)

In [11]:
with open('data/ids_dict.json', 'r', encoding='utf8') as f:
    ids_dict = json.load(f)

In [12]:
len(ids_dict)

1103

## Create subgraph - graph filtering by 'hiponimia'

In [13]:
filtered_edges = [edge for edge in tqdm(list(G.edges)) if edge[2].endswith('hiperonimia')] # if (edge[2].endswith('hiperonimia')) or (edge[2].endswith('hiponimia'))]

100%|██████████| 1438540/1438540 [00:01<00:00, 1386817.19it/s]


In [14]:
subG = nx.DiGraph()
for edge in tqdm(filtered_edges):
    subG.add_edge(edge[0], edge[1], label=edge[2])

100%|██████████| 204667/204667 [00:01<00:00, 158183.82it/s]


In [15]:
print(f'Nodes: {len(subG.nodes)} - part of G: {round(len(subG.nodes)/len(G.nodes), 5)}')
print(f'Edges: {len(subG.edges)} - part of G: {round(len(subG.edges)/len(G.edges), 5)}')

Nodes: 173726 - part of G: 0.50999
Edges: 204667 - part of G: 0.14227


## Removing cycles

In [16]:
cycles = list(simple_cycles(subG))

for cycle in cycles:
    print('\nCYCLE')
    print('ID \t WORD \t\tIN_SIMLEX')
    for node in cycle:
        word = wn.synset_by_id(node).to_dict()['units'][0]['lemma']
        in_simlex = int(node) in list(ids_dict.values())
        print(f'{node} \t {word}  \t{in_simlex}')


CYCLE
ID 	 WORD 		IN_SIMLEX
2373 	 podjąć  	False
2497 	 zająć się  	False
2355 	 zrobić  	True
44782 	 zacząć  	False

CYCLE
ID 	 WORD 		IN_SIMLEX
2496 	 zajmować się  	False
55305 	 robić  	True
2367 	 podejmować  	False


In [17]:
true_node = '2355'
false_nodes = ['2373', '44782', '2497']
for false_node in false_nodes:
    subG = contracted_nodes(subG, true_node, false_node, self_loops=False)

true_node = '55305'
false_nodes = ['2496', '2367']
for false_node in false_nodes:
    subG = contracted_nodes(subG, true_node, false_node, self_loops=False)

In [18]:
len(list(simple_cycles(subG)))

0

## Adding main root node

In [19]:
root_nodes = [k for (k, v) in subG.in_degree() if v == 0]
print(len(root_nodes))

for node in root_nodes:
    subG.add_edge('root', node)

root_nodes = [k for (k, v) in subG.in_degree() if v == 0]
print(len(root_nodes))

4114
1


## Wu and Palmer’s Conceptual Similarity

In [56]:
def wu_palmer(node_1, node_2):
    lso = lowest_common_ancestor(subG, node_1, node_2)

    a = 2 * len(nx.shortest_path(subG, 'root', lso))
    b = len(nx.shortest_path(subG.to_undirected(), node_1, lso)) + len(nx.shortest_path(subG.to_undirected(), node_2, lso)) + a

    return a/b

In [60]:
# TEST
start_tme = time.time()
print(wu_palmer('262143', '1726'))
exec_time = time.time() - start_tme

exec_time

0.8


13.396488904953003

## Leacock and Chodorow’s Normalized Path Length

In [50]:
sp_root = nx.shortest_path_length(subG, 'root')

In [57]:
max_depth = max(sp_root.values()) + 1

max_depth

28

In [61]:
def lc_normalized_path(node_1, node_2):
    a = len(nx.shortest_path(subG.to_undirected(), node_1, node_2))
    b = 2 * max_depth

    return math.log(a/b) * -1

In [59]:
# TEST
start_tme = time.time()
print(lc_normalized_path('262143', '1726'))
exec_time = time.time() - start_tme

exec_time

3.332204510175204


2.09364914894104

## Counting similarities and addig to dataframe

In [26]:
words_in_graph = set(ids_dict.keys())
filtered_simlex = simlex[(simlex['word1'].isin(words_in_graph)) & (simlex['word2'].isin(words_in_graph))]

In [27]:
def add_wp(simlex_row):
    node_1 = str(ids_dict[simlex_row['word1']])
    node_2 = str(ids_dict[simlex_row['word2']])

    try:
        simlex_row['wu_palmer'] = wu_palmer(node_1, node_2)
    except Exception as e:
        print(e)
        simlex_row['wu_palmer'] = None

    return simlex_row


def add_lcn(simlex_row):
    node_1 = str(ids_dict[simlex_row['word1']])
    node_2 = str(ids_dict[simlex_row['word2']])

    try:
        simlex_row['leacon'] = lc_normalized_path(node_1, node_2)
    except Exception as e:
        print(e)
        simlex_row['leacon'] = None

    return simlex_row

In [28]:
filtered_simlex = filtered_simlex.progress_apply(add_wp, axis=1)

100%|██████████| 941/941 [3:07:54<00:00, 11.98s/it]


In [29]:
filtered_simlex = filtered_simlex.progress_apply(add_lcn, axis=1)

100%|██████████| 941/941 [34:08<00:00,  2.18s/it]


In [30]:
filtered_simlex = filtered_simlex.dropna().reset_index(inplace=False)

In [31]:
filtered_simlex.to_csv('out/wordnet_results.csv')

## K top similar words

In [83]:
def get_synset_id(word):
    return str(wn.synsets(word)[0].to_dict()['id'])

In [90]:
def k_top(k, words, metric_fun, max=True):
    
    combs = list(itertools.combinations(words, 2))
    metrics = {}
    for comb in tqdm(combs):
        id_1 = get_synset_id(comb[0])
        id_2 = get_synset_id(comb[1])
        metric = metric_fun(id_1, id_2)
        metrics[str(set(comb))] = metric

    results = {} # comb: metric

    combs = list(itertools.combinations(words, k))
    for comb in combs:
        sub_combs = list(itertools.combinations(comb, 2))
        metrics_to_mean = []
        for sub_comb in sub_combs:
            metrics_to_mean.append(
                metrics[str(set(sub_comb))]
            )
        results[str(comb)] = np.mean(np.array(metrics_to_mean))

    results = {k: v for k, v in sorted(results.items(), key=lambda item: item[1], reverse=max)}

    print()
    print()
    print('TOP 1')
    print('Słowa: ', list(results.items())[0][0])
    print('Wartość metryki: ', round(list(results.items())[0][1], 4))
    print()
    print('TOP 2')
    print('Słowa: ', list(results.items())[1][0])
    print('Wartość metryki: ', round(list(results.items())[1][1], 4))
    print()
    print('TOP 3')
    print('Słowa: ', list(results.items())[2][0])
    print('Wartość metryki: ', round(list(results.items())[2][1], 4))

In [94]:
words = [
    'sufit',
    'pochłonąć',
    'okoliczność',
    'rubież',
    'upraszać',
    'rytm',
    'nowoczesny',
    'pojemnik',
    'gwałtowny',
    'pudełko'
]

### WuPalmer

In [95]:
k_top(2, words, wu_palmer, max=True)

100%|██████████| 45/45 [10:03<00:00, 13.41s/it]

TOP 1
Słowa:  ('sufit', 'rubież')
Wartość metryki:  0.4615

TOP 2
Słowa:  ('rytm', 'pojemnik')
Wartość metryki:  0.2857

TOP 3
Słowa:  ('okoliczność', 'nowoczesny')
Wartość metryki:  0.2



In [96]:
k_top(3, words, wu_palmer, max=True)

100%|██████████| 45/45 [09:31<00:00, 12.71s/it]

TOP 1
Słowa:  ('sufit', 'okoliczność', 'rubież')
Wartość metryki:  0.2751

TOP 2
Słowa:  ('sufit', 'rubież', 'nowoczesny')
Wartość metryki:  0.2751

TOP 3
Słowa:  ('sufit', 'rubież', 'pojemnik')
Wartość metryki:  0.2751



### LeacockChodorow

In [97]:
k_top(2, words, lc_normalized_path, max=True)

100%|██████████| 45/45 [02:10<00:00,  2.90s/it]

TOP 1
Słowa:  ('sufit', 'rubież')
Wartość metryki:  2.4159

TOP 2
Słowa:  ('sufit', 'okoliczność')
Wartość metryki:  2.2336

TOP 3
Słowa:  ('sufit', 'pudełko')
Wartość metryki:  2.2336



In [98]:
k_top(3, words, lc_normalized_path, max=True)

100%|██████████| 45/45 [02:07<00:00,  2.84s/it]

TOP 1
Słowa:  ('sufit', 'rubież', 'pudełko')
Wartość metryki:  2.2944

TOP 2
Słowa:  ('sufit', 'okoliczność', 'rubież')
Wartość metryki:  2.243

TOP 3
Słowa:  ('sufit', 'okoliczność', 'pudełko')
Wartość metryki:  2.1822

